In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd

In [2]:
def get_personal_url(u):

    user_url = urlopen(str(u))
    soup = BeautifulSoup(user_url, 'html.parser')
    
    personal_page_url = []
    for i in soup.find('div', {'id' : 'box01'}).find_all('a', href = True):
        personal_page_url.append(str('https://lis.ly.gov.tw')+ str(i['href']))
    return personal_page_url

In [3]:
u = 'https://lis.ly.gov.tw/lylegismc/lylegismemkmout?!!FUNC400'
personal_url = get_personal_url(u)

In [4]:
del personal_url[101]

In [5]:
del personal_url[124]

## 個人頁

In [ ]:
#問政成果-主頁
res = []
#問政統計
sta = []
for u in personal_url:
    user_url = urlopen(u)
    soup = BeautifulSoup(user_url, 'html.parser')
    
    temp_ = []
    
    
    for i in soup.find('td', class_ = 'person_t02').find_all('a', href = True):
        temp_.append(str('https://lis.ly.gov.tw')+ str(i['href']))
    
    #collect list of url for 問政成果
    res.append(temp_[1])
    #collect list of url for 問政統計
    sta.append(temp_[2])

## 問政成果-委員諮詢

In [ ]:
#問政成果-委員諮詢
res_advisory = []

for u in res:
    user_url = urlopen(u)
    soup = BeautifulSoup(user_url, 'html.parser')
    
    if len(soup.find('div', attrs={"class":"pop","id":"pop02"}).find_all('a', href = True)):
        res_advisory.append(' ')
    else:
        for i in soup.find('div', attrs={"class":"pop","id":"pop02"}).find_all('a', href = True):
            res_advisory.append(str('https://lis.ly.gov.tw')+ str(i['href']))
     

In [ ]:
date_list = []
case_url = []
idx = 0
for u in res_advisory:
    
    if u != ' ':
        user_url = urlopen(u)
        soup = BeautifulSoup(user_url, 'html.parser')

        #質詢日期
        for i in soup.find('table', class_ = 'sumtab').find_all('td', class_ = 'sumtd2000'):
            date_list.append([idx, i.get_text().strip()])

        #案由
        table = soup.find('table', class_ = 'sumtab')
        rows = table.find_all('td', class_ ='sumtd2001')
        for row in rows:
            cols = row.find_all('a')
            for col in cols:
                case_url.append([idx, str('https://lis.ly.gov.tw')+ str(col['href'])]) 
    else:

        date_list.append('')
        case_url.append('')
    idx += 1

In [ ]:
print(len(date_list))
print(len(case_url))

## Merge data

In [ ]:
a = pd.DataFrame(date_list, columns = ['idx', 'date'])
b = pd.DataFrame(case_url, columns = ['idx1', 'case_url'])

In [ ]:
c = pd.merge(a, b, left_index= True, right_index= True)

In [ ]:
c.set_index('idx')

In [ ]:
df = pd.DataFrame({'main_page_url': personal_url, 'result_url': res, 'result_advisory_url': res_advisory, 'statistic': sta})

In [ ]:
df.reset_index(inplace= True)

In [ ]:
df_f = df.merge(c, left_on = 'index', right_on = 'idx' )

In [ ]:
df_f.head()

In [ ]:
df_f.columns

In [ ]:
del df_f['idx']
del df_f['idx1']

In [ ]:
df_f.to_csv()

## 問政成果-詳細資料

## 手動輸入

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import requests

In [ ]:
sec_list = []

In [10]:
# 輸入url
u = ''

In [32]:
temp_url = []
user_url = urlopen(u)
soup = BeautifulSoup(user_url, 'html.parser')

table = soup.find('td', attrs={'class':'disp62_m'})
temp = table.find('td', attrs={'class':'dettb02'}).get_text()


#name
try:
    name = temp.split('\n')[0]
except IndexError:
    name = ' '

#property
try:
    propty = temp.split('\n')[1][4:]
except IndexError:
    property = ' '

#inquiry
try:
    inqry =  temp.split('\n')[2][2:]
except IndexError:
    inqry = ' '

# case 
try:
    case = temp.split('\n')[3][2:]
except IndexError:
    case = ' '

#reply_check
try:
    reply_chk = temp.split('\n')[4][3:]
except IndexError:
    reply_chk = ' '

    #reply
try:
    reply = temp.split('\n')[5][2:]
except IndexError:
    reply = ' '

#people_reply
try:
    ppl_reply = temp.split('\n')[6][3:]
except IndexError:
    ppl_reply = ' '

#date_inquiry
try:
    date_inqry = temp.split('\n')[7][4:]
except IndexError:
    date_inqry = ' '

#reply date
try:
    date_reply = temp.split('\n')[8][4:]
except IndexError:
    date_reply = ' '

#system no
try:
    sys_no = temp.split('\n')[9][3:]
except IndexError:
    sys_no = ' '

#category
try:
    ctgry = temp.split('\n')[10][2:]
except IndexError:
    ctgry = ' '

#keyword
try:
    keyword = temp.split('\n')[11][3:]
except IndexError:
    keyword = ' '

#pdf link
try:
    link_ = table.find_all('a', attrs = {'title':"PDF文字型電子檔"}, href = True)[0]
    pdf_link = str('https://lis.ly.gov.tw')+ str(link_['href'])
except IndexError:    
    pdf_link = ''

    
col_header =['name', 'property', 'inquiry', 'case', 'reply_check', 'reply', 'people_reply', 'date_inquiry', 'date_reply', 'sys_no',
      'category', 'keyword', 'pdf_link'] 
    
sec_list.append([name, propty, inqry, case, reply_chk, reply, ppl_reply, date_inqry, date_reply, sys_no,
      ctgry, keyword, pdf_link])
temp_url = pdf_link

In [22]:


def download_pdf(url, path, fn):
    '''
    url the url link for pdf
    path to save the file to e.g. /Docs/test.pdf
    '''
    r = requests.get(url, stream=True)
    chunk_size = 24
    if path == None:
        _path = fn + '.pdf'
    else:
        _path = os.path.join(path, fn + '.pdf')
        
    print (_path)
    with open(_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size):
            fd.write(chunk)



In [37]:

download_pdf(temp_url, None,  'test')


test.pdf


In [34]:
temp_url

'https://lis.ly.gov.tw/lylegisc/lylegiskmout?000889AB0014010300000000000501400000003D000000000^IMG_lypdftxt_1^xdd!cecac8cccbcfcbc9cacd81cecfc8cfc7cdcfcec4cfcfc8c8c4cfcfc7cf'

In [ ]:
df = pd.DataFrame(sec_list, columns = col_header)

### =================截止==================

In [ ]:
sec_list = []

for u in df_f['case_url']:
    user_url = urlopen(u)
    soup = BeautifulSoup(user_url, 'html.parser')

    table = soup.find('td', attrs={'class':'disp62_m'})
    temp = table.find('td', attrs={'class':'dettb02'}).get_text()


    #name
    try:
        name = temp.split('\n')[0]
    except IndexError:
        name = ' '

    #property
    try:
        propty = temp.split('\n')[1][4:]
    except IndexError:
        property = ' '

    #inquiry
    try:
        inqry =  temp.split('\n')[2][2:]
    except IndexError:
        inqry = ' '

    # case 
    try:
        case = temp.split('\n')[3][2:]
    except IndexError:
        case = ' '

    #reply_check
    try:
        reply_chk = temp.split('\n')[4][3:]
    except IndexError:
        reply_chk = ' '

        #reply
    try:
        reply = temp.split('\n')[5][2:]
    except IndexError:
        reply = ' '

    #people_reply
    try:
        ppl_reply = temp.split('\n')[6][3:]
    except IndexError:
        ppl_reply = ' '

    #date_inquiry
    try:
        date_inqry = temp.split('\n')[7][4:]
    except IndexError:
        date_inqry = ' '

    #reply date
    try:
        date_reply = temp.split('\n')[8][4:]
    except IndexError:
        date_reply = ' '

    #system no
    try:
        sys_no = temp.split('\n')[9][3:]
    except IndexError:
        sys_no = ' '

    #category
    try:
        ctgry = temp.split('\n')[10][2:]
    except IndexError:
        ctgry = ' '

    #keyword
    try:
        keyword = temp.split('\n')[11][3:]
    except IndexError:
        keyword = ' '

    #pdf link
    try:
        link_ = table.find_all('a', attrs = {'title':"PDF文字型電子檔"}, href = True)[0]
        pdf_link = str('https://lis.ly.gov.tw')+ str(link_['href'])
    except IndexError:
        pdf_link = ' '
        
    sec_list.append([name, propty, inqry, case, reply_chk, reply, ppl_reply, date_inqry, date_reply, sys_no,
          ctgry, keyword, pdf_link])
    

In [ ]:
sec_list

In [ ]:
df_sec = pd.DataFrame(sec_list)

In [ ]:
df_sec.to_csv('data.csv')

## 提案類別統計

In [ ]:
u = 'https://lis.ly.gov.tw/lylegisc/lylegiskmout?.066300A7000000000001000^000000E0000E1000000000000074713eb5'
user_url = urlopen(u)
soup = BeautifulSoup(user_url, 'html.parser')

In [ ]:
table = soup.find('table', attrs = {'class': 'column_list', 'id': 'lgmempro_class_data' })

In [ ]:
industry = []
for i in table.find_all('th'):
    industry.append(i.get_text())

In [ ]:
num = []
for i in table.find_all('td'):
    num.append(i.get_text())

In [ ]:
pd.DataFrame(list(zip(industry, num)))

## 進度統計

In [ ]:
table = soup.find('table', attrs = {'class': 'pie_list', 'id': 'lgmempro_progress_data' })

In [ ]:
progress = []
for i in table.find_all('td', attrs = {'class':'', 'style': ''}):
    progress.append(i.get_text())

In [ ]:
progress

In [ ]:
count = []
for i in table.find_all('td', attrs = {'class':'total'}):
    count.append(i.get_text())

In [ ]:
len(count)